In [7]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import gc

import warnings
import time
import sys
import datetime

import holidays
holidays = holidays.Brazil()

warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [8]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [9]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [10]:
def read_data(input_file):
    df = pd.read_csv(input_file, parse_dates=['first_active_month'], nrows = 1000)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    return df
#_________________________________________
train = read_data('C:/Users/1/Python/Kaggle/Elo loyalty card - 2/input/train.csv')
test = read_data('C:/Users/1/Python/Kaggle/Elo loyalty card - 2/input/test.csv')
hist_trans = reduce_mem_usage(pd.read_csv('C:/Users/1/Python/Kaggle/Elo loyalty card - 2/input/historical_transactions.csv', nrows = 10000))
new_merchant_trans = reduce_mem_usage(pd.read_csv('C:/Users/1/Python/Kaggle/Elo loyalty card - 2/input/new_merchant_transactions.csv', nrows = 10000))

Mem. usage decreased to  0.57 Mb (46.4% reduction)
Mem. usage decreased to  0.57 Mb (46.4% reduction)


In [11]:
for df in [hist_trans,new_merchant_trans]:
    df['category_2'].fillna(1.0,inplace=True)
    df['category_3'].fillna('A',inplace=True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)

for df in [hist_trans,new_merchant_trans]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    #https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    
hist_trans = pd.get_dummies(hist_trans, columns=['category_3'])
new_merchant_trans = pd.get_dummies(new_merchant_trans, columns=['category_3'])

In [12]:
hist_trans_autorized = hist_trans[hist_trans['authorized_flag']==1]

gdf = hist_trans.groupby("card_id")
gdf = gdf["authorized_flag"].mean().reset_index()
gdf.columns = ["card_id", "authorized_flag_mean"]
df_train = pd.merge(train, gdf, on="card_id", how="left")
df_test = pd.merge(test, gdf, on="card_id", how="left")

del gdf
del hist_trans
gc.collect()

410

In [13]:
gdf = hist_trans_autorized.groupby('card_id')['installments'].agg(lambda x: x.value_counts().index[0])
gdf = gdf.reset_index()
gdf.columns = ["card_id", "installments"]
df_train = pd.merge(df_train, gdf, on="card_id", how="left")
df_test = pd.merge(df_test, gdf, on="card_id", how="left")
del gdf; gc.collect()

35

In [14]:
gdf = hist_trans_autorized.groupby('card_id')['city_id'].agg(lambda x: x.value_counts().index[0])
gdf = gdf.reset_index()
gdf.columns = ["card_id", "city_id"]
df_train = pd.merge(df_train, gdf, on="card_id", how="left")
df_test = pd.merge(df_test, gdf, on="card_id", how="left")
del gdf; gc.collect()

70

In [15]:
gdf = hist_trans_autorized.groupby('card_id')['state_id'].agg(lambda x: x.value_counts().index[0])
gdf = gdf.reset_index()
gdf.columns = ["card_id", "state_id"]
df_trainf = pd.merge(df_train, gdf, on="card_id", how="left")
df_test = pd.merge(df_test, gdf, on="card_id", how="left")
del gdf; gc.collect()

63

In [16]:
gdf = hist_trans_autorized.groupby('card_id')['category_2'].agg(lambda x: x.value_counts().index[0])
gdf = gdf.reset_index()
gdf.columns = ["card_id", "category_2"]
df_train = pd.merge(df_train, gdf, on="card_id", how="left")
df_test = pd.merge(df_test, gdf, on="card_id", how="left")
del gdf; gc.collect()

70

In [17]:
aggs = {}
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']
    
aggs['purchase_amount'] = ['sum','max','min','mean']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean']
aggs['weekend'] = ['mean']
aggs['category_1'] = ['mean']
aggs['card_id'] = ['size']
aggs['category_3_A'] = ['mean']
aggs['category_3_B'] = ['mean']
aggs['category_3_C'] = ['mean']

new_columns = get_new_columns('hist',aggs)
df_hist_trans_group = hist_trans_autorized.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['hist_purchase_date_diff'] = (df_hist_trans_group['hist_purchase_date_max'] - df_hist_trans_group['hist_purchase_date_min']).dt.days
df_hist_trans_group['hist_purchase_date_average'] = df_hist_trans_group['hist_purchase_date_diff']/df_hist_trans_group['hist_card_id_size']
df_hist_trans_group['hist_purchase_date_uptonow'] = (datetime.datetime.today() - df_hist_trans_group['hist_purchase_date_max']).dt.days
df_hist_trans_group = reduce_mem_usage(df_hist_trans_group)
df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group;gc.collect()

Mem. usage decreased to  0.00 Mb (64.7% reduction)


56

In [18]:
aggs = {}
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']
    
aggs['purchase_amount'] = ['sum','max','min','mean']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean']
aggs['weekend'] = ['mean']
aggs['category_1'] = ['mean']
aggs['card_id'] = ['size']
aggs['category_3_A'] = ['mean']
aggs['category_3_B'] = ['mean']
aggs['category_3_C'] = ['mean']

new_columns = get_new_columns('new',aggs)
df_new_trans_group = new_merchant_trans.groupby('card_id').agg(aggs)
df_new_trans_group.columns = new_columns
df_new_trans_group.reset_index(drop=False,inplace=True)
df_new_trans_group['new_purchase_date_diff'] = (df_new_trans_group['new_purchase_date_max'] - df_new_trans_group['new_purchase_date_min']).dt.days
df_new_trans_group['new_purchase_date_average'] = df_new_trans_group['new_purchase_date_diff']/df_new_trans_group['new_card_id_size']
df_new_trans_group['new_purchase_date_uptonow'] = (datetime.datetime.today() - df_new_trans_group['new_purchase_date_max']).dt.days
df_new_trans_group = reduce_mem_usage(df_new_trans_group)
df_train = df_train.merge(df_new_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_new_trans_group,on='card_id',how='left')
del df_new_trans_group;gc.collect()

Mem. usage decreased to  0.05 Mb (67.0% reduction)


49

In [21]:
del hist_trans_autorized;gc.collect()
del new_merchant_trans;gc.collect()


NameError: name 'hist_trans_autorized' is not defined

In [23]:
for df in [df_train,df_test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_purchase_date_max',\
                     'new_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
    df['card_id_total'] = df['hist_card_id_size']+df['new_card_id_size']
    df['purchase_amount_total'] = df['hist_purchase_amount_sum']+df['new_purchase_amount_sum']
#all_col = list(df_train.columns)

In [ ]:
df_train = pd.DataFrame(df_train)
df_train.to_csv("df_train.csv", index=False)
df_test = pd.DataFrame(df_train)
df_test.to_csv("df_test.csv", index=False)